In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
detector = HandDetector(maxHands=2)  # Detect up to 2 hands
offset = 20
imgSize = 300
counter_left = 0
counter_right = 0

# Folders to save left and right hand data
folder_left = r"C:\Users\amank\OneDrive\Desktop\Sign-Language-detection-main\Data2\Left_Hand"
folder_right = r"C:\Users\amank\OneDrive\Desktop\Sign-Language-detection-main\Data2\Right_Hand"

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        continue

    # Resize the image to reduce processing load
    img = cv2.resize(img, (640, 480))

    hands, img = detector.findHands(img)
    if hands:
        for i, hand in enumerate(hands):
            x, y, w, h = hand['bbox']
            handType = hand['type']  # Determine if it's the left or right hand

            print(f"Hand {i + 1}: {handType}, x: {x}, y: {y}, w: {w}, h: {h}")
            if y - offset < 0 or y + h + offset > img.shape[0] or x - offset < 0 or x + w + offset > img.shape[1]:
                print("ROI is out of bounds")
                continue

            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
            if imgCrop.size == 0:
                print("Cropped image is empty")
                continue

            aspectRatio = h / w

            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap: wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap: hCal + hGap, :] = imgResize

            cv2.imshow(f'ImageCrop_{handType}', imgCrop)
            cv2.imshow(f'ImageWhite_{handType}', imgWhite)

            # Save the image for the corresponding hand
            key = cv2.waitKey(1)
            if key == ord("s"):
                if handType == "Left":
                    counter_left += 1
                    cv2.imwrite(f'{folder_left}/Image_{time.time()}.jpg', imgWhite)
                    print(f"Left hand image saved: {counter_left}")
                elif handType == "Right":
                    counter_right += 1
                    cv2.imwrite(f'{folder_right}/Image_{time.time()}.jpg', imgWhite)
                    print(f"Right hand image saved: {counter_right}")

    cv2.imshow('Image', img)

    # Check for the 'ESC' key to exit
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
